# University of Michigan
## School of Information
**Masters of Applied Data Science**<br>
**Milestone 1**<br>
<br>
Affordable Housing Project
<br><br>
**Submitted by**<br>
>Alan Fehsenfeld<br>Koon Leong Ho<br>George Thio

**Data Preparation**

In [1]:
import sqlalchemy
import pandas as pd
import geopandas
import numpy as np
from config import dbconfig, basedir

properties_input = r'/data/property_level.csv'
properties_output=r'/data/SIADS591_property.pkl'

properties_shp_file = r'/data/municipalities/All Municipalities Geometry and Municipality.shp'
properties_gis_output = r'/data/SIADS591municipalities.feather'

# a flag used to reduce the number of properties processed to save time during development
test_mode = False

In [2]:
# Upload property csv file into database


def convert2int(df):
    return df.astype('int')

def calculate_percentage(data):
    # calculates the percentage each plot will contribute to the total number of properties
    # in the municipalities. 
    # It is, essentially, the inverse of the number of plots multiplied by 100 to derive
    # the percentage contribution by each plot.  The sum of all properties within a given 
    # municipalities will add up to 100.
    #
    list_of_cities = list(data['City'].unique())
    p_data = data.copy()
    p_data['Percent']=0
    for city in list_of_cities:
        total_in_city = len(p_data[p_data.City == city])
        p_data.loc[p_data.City == city, ['Percent']] = (1/total_in_city)*100
    return p_data

def prep_properties():
    file_dir = basedir()
    full_path = file_dir + properties_input
    print(f'Data file {full_path}')
    df = pd.read_csv(full_path)
    df = df.iloc[:,2:-1]
    df = df.rename(columns = {'PNUM': 'APN'})
    df['City'] = df['City'].str.upper()
    df = df.fillna(0)
    df = df[~(df.City==0)]
    df['Per Arce'] = convert2int(df['SEV']/df['Acres'])
    df['SEV'] = convert2int(df['SEV'])
    df['MSHDA Score'] = convert2int(df['MSHDA Score'])
    df['amenities_ind'] = convert2int(df['amenities_ind'])
    df['walkscore_ind'] = convert2int(df['walkscore_ind'])
    
    df = calculate_percentage(df)
    
    full_path = file_dir + properties_output
    if test_mode:
        df = df.iloc[0:100,:]
        df.to_pickle(full_path)        
    else:
        df.to_pickle(full_path)
    return df


**The  code in the next cell has to be uncommented to run the data preparation.**


In [3]:
# prep_properties()